# 1

pip install mysql-connector-python


In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import mysql.connector
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Load Iris dataset
iris = pd.read_csv("Iris2023.csv")
x = iris.drop("species", axis=1)
y = iris["species"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

app = dash.Dash(__name__)

# Database setup
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='iris'
)
cursor = conn.cursor()

# Create and fit the KNN model
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)

# Create a table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS user_inputs (
        id INT AUTO_INCREMENT PRIMARY KEY,
        sepal_length DOUBLE,
        sepal_width DOUBLE,
        petal_length DOUBLE,
        petal_width DOUBLE
    )
''')
conn.commit()

app.layout = html.Div([
    html.H1("Iris Flower Prediction", style={'textAlign': 'center', 'marginBottom': 20, 'color': '#333'}),

    html.Label("Enter Sepal Length:"),
    dcc.Input(id='sepal_length', type='number', value=5.1),

    html.Label("Enter Sepal Width:"),
    dcc.Input(id='sepal_width', type='number', value=3.5),

    html.Label("Enter Petal Length:"),
    dcc.Input(id='petal_length', type='number', value=1.4),

    html.Label("Enter Petal Width:"),
    dcc.Input(id='petal_width', type='number', value=0.2),

    html.Button(id='store_button', n_clicks=0, children='Store Input'),

    html.Div(id='output_message', style={'marginTop': 20, 'fontSize': 18}),
])

@app.callback(
    Output('output_message', 'children'),
    [Input('store_button', 'n_clicks')],
    [
        State('sepal_length', 'value'),
        State('sepal_width', 'value'),
        State('petal_length', 'value'),
        State('petal_width', 'value')
    ]
)
def predict_and_store(n_clicks, sepal_length, sepal_width, petal_length, petal_width):
    if n_clicks > 0 and all(v is not None for v in [sepal_length, sepal_width, petal_length, petal_width]):
        # Store input values in the database
        cursor.execute('''
            INSERT INTO user_inputs (sepal_length, sepal_width, petal_length, petal_width)
            VALUES (%s, %s, %s, %s)
        ''', (sepal_length, sepal_width, petal_length, petal_width))
        conn.commit()

        # Predict
        x_new = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
        prediction = knn.predict(x_new)[0]

        # Prediction probability scores
        prob_scores = knn.predict_proba(x_new)

        return f'Predicted Species: {prediction}, Prediction Score: {prob_scores.max():.2f}, Input values stored in the Iris database'

    elif n_clicks > 0:
        return 'Please enter all values to get a prediction'

    return ''

if __name__ == '__main__':
    app.run_server(port=8056, debug=True)


# 4

In [16]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import mysql.connector
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

# Load Iris dataset
iris = pd.read_csv("Iris2023.csv")
x = iris.drop("species", axis=1)
y = iris["species"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

app = dash.Dash(__name__)

# Database setup
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='iris'
)
cursor = conn.cursor()

# Create and fit the KNN model
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)

# Create a table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS iris_predictions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        sepal_length DOUBLE,
        sepal_width DOUBLE,
        petal_length DOUBLE,
        petal_width DOUBLE,
        predicted_species VARCHAR(255),
        prediction_score DOUBLE
    )
''')
conn.commit()

app.layout = html.Div([
    html.H1("Iris Flower Prediction", style={'textAlign': 'center', 'color': 'darkviolet', 'fontFamily': 'Arial'}),

    html.Div([
        html.Label("Enter Sepal Length:", style={'marginRight': '10px', 'textAlign': 'center'}),
        dcc.Input(id='sepal_length', type='number', value=5.1, style={'marginRight': '10px', 'textAlign': 'center'}),
    ], style={'marginBottom': '10px', 'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        html.Label("Enter Sepal Width:", style={'marginRight': '10px', 'textAlign': 'center'}),
        dcc.Input(id='sepal_width', type='number', value=3.5, style={'marginRight': '10px', 'textAlign': 'center'}),
    ], style={'marginBottom': '10px', 'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        html.Label("Enter Petal Length:", style={'marginRight': '10px', 'textAlign': 'center'}),
        dcc.Input(id='petal_length', type='number', value=1.4, style={'marginRight': '10px', 'textAlign': 'center'}),
    ], style={'marginBottom': '10px', 'display': 'flex', 'justifyContent': 'center'}),

    html.Div([
        html.Label("Enter Petal Width:", style={'marginRight': '10px', 'textAlign': 'center'}),
        dcc.Input(id='petal_width', type='number', value=0.2, style={'marginRight': '10px', 'textAlign': 'center'}),
    ], style={'marginBottom': '20px', 'display': 'flex', 'justifyContent': 'center'}),

    html.Button(id='store_button', n_clicks=0, children='Species', style={'marginBottom': '20px','font-weight': 'bold', 'background-color': '#007BFF', 'color': 'white', 'align-self': 'center'}),

    html.Div(id='output_message', style={'marginTop': '20px', 'fontSize': '18px', 'color': 'red','font-weight': 'bold', 'fontFamily': 'Arial','textAlign': 'center'}),
], style={'backgroundImage': 'url("https://wallpapercave.com/wp/qqGZC6u.jpg")', 'backgroundSize': 'cover'})

@app.callback(
    Output('output_message', 'children'),
    [Input('store_button', 'n_clicks')],
    [
        State('sepal_length', 'value'),
        State('sepal_width', 'value'),
        State('petal_length', 'value'),
        State('petal_width', 'value')
    ]
)
def predict_and_store(n_clicks, sepal_length, sepal_width, petal_length, petal_width):
    if n_clicks > 0 and all(v is not None for v in [sepal_length, sepal_width, petal_length, petal_width]):
        # Store input values in the database
        cursor.execute('''
            INSERT INTO iris_predictions (sepal_length, sepal_width, petal_length, petal_width)
            VALUES (%s, %s, %s, %s)
        ''', (sepal_length, sepal_width, petal_length, petal_width))
        conn.commit()

        # Predict
        x_new = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
        predicted_species = knn.predict(x_new)[0]

        # Prediction probability scores
        prediction_score = knn.predict_proba(x_new).max()

        # Update the prediction in the database
        cursor.execute('''
            UPDATE iris_predictions
            SET predicted_species = %s, prediction_score = %s
            ORDER BY id DESC
            LIMIT 1
        ''', (predicted_species, prediction_score))
        conn.commit()

        return f'Predicted Species: {predicted_species}, Prediction Score: {prediction_score:.2f}, Input values stored in the database!'

    elif n_clicks > 0:
        return 'Please enter all values to get a prediction'

    return ''

if __name__ == '__main__':
    app.run_server(port=8057, debug=True, external_stylesheets=external_stylesheets)


# 5